In [29]:
using DifferentialEquations, Serialization
include("ode_problem.jl")
include("calculate_experimental_outputs.jl")
include("make_predictions.jl");

Specify Run of Interest

In [30]:
#run = "291";

Create ODE Problems for each initial condition

In [31]:
#ode problems
normalize_to = normalize_to_dose()
ligand_doses = cat(return_ligand_dose_nM(),normalize_to, dims=1) #normalize to last stimulation

n_doses = length(ligand_doses)
#next input we need is an ode problem
odeproblems = []
for l in ligand_doses
    odesys, u0, tspan, p = return_ode_problem_default_inputs(l)
    if l != 1000.0
        tspan = (0,65)
    end
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p)
    push!(odeproblems, prob)
end

#odeproblems for finegrain dose response
ligand_doses = return_finegrain_ligand_dose_nM()
n_doses_finegrain = length(ligand_doses)

odeproblems_finegrain = []
for l in ligand_doses
    odesys, u0, tspan, p = return_ode_problem_default_inputs(l)
    if l != 1000.0
        tspan = (0,65)
    end
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p)
    push!(odeproblems_finegrain, prob)
end

Load ODE Solver Inputs

In [32]:
#need ode solver inputs
#save at changes based on output
odesolver_inputs_timecourse = return_ode_problem_solver_default_inputs("timecourse")
odesolver_inputs_dose_response = return_ode_problem_solver_default_inputs("dose response")
save_at_finegrain = return_finegrain_saveat()
odesolver_inputs_timecourse_finegrain = copy(odesolver_inputs_timecourse )
odesolver_inputs_timecourse_finegrain["saveat"] = save_at_finegrain;

In [33]:
#change solver tolerances for very low ligand dose
#this dose has no experimental value, it's only for plotting
odesolver_inputs_dose_response["reltol"] = 1.0e-12
odesolver_inputs_dose_response["abstol"] = 1.0e-9;

Define Type of ODE Solution

In [34]:
#get type of ODEProblem solution
pred = DifferentialEquations.solve(odeproblems[1], odesolver_inputs_timecourse["solver"], abstol=odesolver_inputs_timecourse["abstol"], reltol=odesolver_inputs_timecourse["reltol"], saveat=odesolver_inputs_timecourse["saveat"])
ode_sol_type = typeof(pred);

Load posterior samples and convert from log10 scale

In [35]:
parameters = 10.0.^deserialize("outputs/300_$(run)_posterior_samples_thinned.jls")
n_parameters, n_samples = size(parameters);

Simulate Timecourse using Posterior Samples

In [36]:
predicted = Array{ode_sol_type}(undef, n_samples)
predicted_finegrain = Array{ode_sol_type}(undef, n_samples)
for j in 1:n_samples
    predicted[j] = make_prediction(odeproblems[end], odesolver_inputs_timecourse, parameters[:,j]) #odeproblem[end] corresponds to ligand = 1000 nM
    predicted_finegrain[j] = make_prediction(odeproblems[end], odesolver_inputs_timecourse_finegrain, parameters[:,j])
end

Calculate Experimental Quantities (here, only active G protein) & Save

In [37]:
timecourse_active_g = [calculate_all_quantities(predicted[i])["fraction_active_G"] for i in 1:n_samples]
timecourse_active_g_finegrain = [calculate_all_quantities(predicted_finegrain[i])["fraction_active_G"] for i in 1:n_samples]
serialize("outputs/500_$(run)_predictions_active_g_timecourse.jls", timecourse_active_g)
serialize("outputs/500_$(run)_finegrain_predictions_active_g_timecourse.jls", timecourse_active_g_finegrain);

Simulate Dose Response using Posterior Samples

In [ ]:
predicted = Matrix{ode_sol_type}(undef, n_doses, n_samples)
for i in 1:n_doses
    for j in 1:n_samples
        predicted[i,j] = make_prediction(odeproblems[i], odesolver_inputs_dose_response, parameters[:,j])
    end
end

predicted_finegrain = Matrix{ode_sol_type}(undef, n_doses_finegrain, n_samples)
for i in 1:n_doses_finegrain
    for j in 1:n_samples
        predicted_finegrain[i,j] = make_prediction(odeproblems_finegrain[i], odesolver_inputs_dose_response, parameters[:,j])
    end
end

Calculate Experimental Quantities (here, active G protein and RL) & Save

In [39]:
active_g = Matrix{Float64}(undef, n_doses, n_samples)
rl = Matrix{Float64}(undef, n_doses, n_samples)
for i in 1:n_doses
    for j in 1:n_samples
        fraction_activated = calculate_all_quantities(predicted[i,j])
        active_g[i,j] = fraction_activated["fraction_active_G"][1] #need to index vector of 1d
        rl[i,j] = fraction_activated["RL"][1]
    end
end

active_g_finegrain = Matrix{Float64}(undef, n_doses_finegrain, n_samples)
rl_finegrain = Matrix{Float64}(undef, n_doses_finegrain, n_samples)
for i in 1:n_doses_finegrain
    for j in 1:n_samples
        fraction_activated = calculate_all_quantities(predicted_finegrain[i,j])
        active_g_finegrain[i,j] = fraction_activated["fraction_active_G"][1] #need to index vector of 1d
        rl_finegrain[i,j] = fraction_activated["RL"][1]
    end
end

#normalize to dose of 1000 nM for each parameter saveat
for j in 1:n_samples
    active_g[:,j] = active_g[:,j]./active_g[end,j]
    rl[:,j] = rl[:,j]./rl[end,j]
end

for j in 1:n_samples
    active_g_finegrain[:,j] = active_g_finegrain[:,j]./active_g_finegrain[end,j]
    rl_finegrain[:,j] = rl_finegrain[:,j]./rl_finegrain[end,j]
end

serialize("outputs/500_$(run)_predictions_active_g_dose_response.jls", active_g)
serialize("outputs/500_$(run)_finegrain_predictions_active_g_dose_response.jls", active_g_finegrain)
serialize("outputs/500_$(run)_predictions_rl_dose_response.jls", rl)
serialize("outputs/500_$(run)_finegrain_predictions_rl_dose_response.jls", rl_finegrain);

Simulate All Timecourses for all Doses

In [ ]:
#must change tspan for dose response inputs
normalize_to = normalize_to_dose()
ligand_doses = cat(return_ligand_dose_nM(),normalize_to, dims=1) #normalize to last stimulation

n_doses = length(ligand_doses)
#next input we need is an ode problem
odeproblems = []
for l in ligand_doses
    odesys, u0, tspan, p = return_ode_problem_default_inputs(l)
    prob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p)
    push!(odeproblems, prob)
end

In [ ]:
predicted = Matrix{ode_sol_type}(undef, n_doses, n_samples)
predicted_finegrain = Matrix{ode_sol_type}(undef, n_doses, n_samples)
for i in 1:n_doses
    for j in 1:n_samples
        predicted[i,j] = make_prediction(odeproblems[i], odesolver_inputs_timecourse, parameters[:,j])
        predicted_finegrain[i,j] = make_prediction(odeproblems[i], odesolver_inputs_timecourse_finegrain, parameters[:,j])
    end
end
serialize("outputs/500_$(run)_predictions.jls", predicted)
serialize("outputs/500_$(run)_predictions_finegrain.jls", predicted_finegrain)